# GEEMap

In [4]:
#!pip uninstall ee -y
!pip uninstall earthengine-api geemap -y

Found existing installation: earthengine-api 1.7.1
Uninstalling earthengine-api-1.7.1:
  Successfully uninstalled earthengine-api-1.7.1
Found existing installation: geemap 0.36.6
Uninstalling geemap-0.36.6:
  Successfully uninstalled geemap-0.36.6


In [5]:
!pip install earthengine-api geemap

  Using cached earthengine_api-1.7.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached geemap-0.36.6-py3-none-any.whl.metadata (14 kB)
Using cached earthengine_api-1.7.1-py3-none-any.whl (468 kB)
Using cached geemap-0.36.6-py3-none-any.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [geemap]2m1/2 [geemap]


In [2]:
import ee
import geemap
import os

# --- 1. ROBUST AUTHENTICATION FOR WSL ---
try:
    # Try to initialize. If it fails, trigger authentication.
    ee.Initialize()
    print("Google Earth Engine initialized successfully.")
except Exception as e:
    print("Authentication required. Follow the steps in the console...")
    # This will trigger the flow where you copy-paste the token
    ee.Authenticate()
    ee.Initialize()

Authentication required. Follow the steps in the console...


/bin/xdg-open: 882: x-www-browser: not found
/bin/xdg-open: 882: firefox: not found
/bin/xdg-open: 882: iceweasel: not found
/bin/xdg-open: 882: seamonkey: not found
/bin/xdg-open: 882: mozilla: not found
/bin/xdg-open: 882: epiphany: not found
/bin/xdg-open: 882: konqueror: not found
/bin/xdg-open: 882: chromium: Permission denied
/bin/xdg-open: 882: chromium-browser: not found
/bin/xdg-open: 882: google-chrome: not found
/bin/xdg-open: 882: www-browser: not found
/bin/xdg-open: 882: links2: not found
/bin/xdg-open: 882: elinks: not found
/bin/xdg-open: 882: links: not found
/bin/xdg-open: 882: lynx: not found
/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=r5qTJ0uDT-YLSevCylpW3FYX7UWyYsHWxU8xEI


Successfully saved authorization token.


In [2]:
import ee
print(ee.__file__)

None


Now we import data

In [3]:
# --- 2. DATA PROCESSING ---
# Define ROI (Region of Interest) - Example: Central Park, NY
roi = ee.Geometry.Point([-73.9654, 40.7829])

# Load Sentinel-2 Image
# filtering for low clouds in Summer 2023
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
         .filterBounds(roi)
         .filterDate('2023-06-01', '2023-09-30')
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
         .median())

# Calculate NDVI
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Calculate NDWI (Water Index)
ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')

# --- 3. VISUALIZATION (ADJUSTED FOR LOCAL) ---
# Create the map object
Map = geemap.Map(center=[40.7829, -73.9654], zoom=13)

# Define visual parameters
vis_rgb = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}
vis_ndvi = {'min': 0, 'max': 0.8, 'palette': ['blue', 'white', 'green']}

# Add layers
Map.addLayer(image, vis_rgb, 'Sentinel-2 RGB')
Map.addLayer(ndvi, vis_ndvi, 'NDVI')

# --- 4. EXPORT FOR VIEWING ---
# Since you are in WSL, a pop-up window might not work well.
# We will save the interactive map as an HTML file.
output_file = "ndvi_map.html"
Map.to_html(output_file)

print(f"Success! Map saved to: {os.path.abspath(output_file)}")
print("Open this file in your Windows browser to interact with the data.")

Success! Map saved to: /home/mengyuwsl/python_GEE_playground/python_GEE_playground/ndvi_map.html
Open this file in your Windows browser to interact with the data.


In [4]:
winter_img = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate('2023-01-01', '2023-03-01') # Winter dates
              .sort('CLOUDY_PIXEL_PERCENTAGE')        # Get least cloudy
              .first())                               # Just grab the best single image

# Image B: SUMMER (For Built-Up Index)
# Urban areas contrast better against green vegetation
summer_img = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate('2023-06-01', '2023-09-01')
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
              .median())

# --- 3. CALCULATE INDICES ---

# A. NDSI (Normalized Difference Snow Index)
# Formula: (Green - SWIR1) / (Green + SWIR1)
# Sentinel-2: Green = B3, SWIR1 = B11
ndsi = winter_img.normalizedDifference(['B3', 'B11']).rename('NDSI')

# B. NDBI (Normalized Difference Built-Up Index)
# Formula: (SWIR1 - NIR) / (SWIR1 + NIR)
# Sentinel-2: SWIR1 = B11, NIR = B8
ndbi = summer_img.normalizedDifference(['B11', 'B8']).rename('NDBI')

# --- 4. VISUALIZATION ---

Map = geemap.Map(center=[40.7829, -73.9654], zoom=12)

# Vis Params
# Snow: Blue (low snow) to White (high snow)
vis_ndsi = {'min': -0.5, 'max': 0.8, 'palette': ['black', 'cyan', 'white']}

# Built-Up: Green (Veg) to Grey/Red (Urban)
vis_ndbi = {'min': -0.5, 'max': 0.5, 'palette': ['green', 'white', 'red']}

# Add Layers
Map.addLayer(winter_img, {'min':0, 'max':3000, 'bands':['B4','B3','B2']}, 'Winter RGB')
Map.addLayer(ndsi, vis_ndsi, 'NDSI (Snow Index)')

Map.addLayer(summer_img, {'min':0, 'max':3000, 'bands':['B4','B3','B2']}, 'Summer RGB')
Map.addLayer(ndbi, vis_ndbi, 'NDBI (Built-Up Index)')

# --- 5. EXPORT TO HTML ---
output_file = "nyc_snow_urban_analysis.html"
Map.to_html(output_file)

print(f"Map saved to: {os.path.abspath(output_file)}")

Map saved to: /home/mengyuwsl/python_GEE_playground/python_GEE_playground/nyc_snow_urban_analysis.html
